# Detect Trucks Sentinel-2

This Jupyter Notebook presents the workflow developped by Henrick Fisser (henrik.fisser@t-online.de) to detect moving trucks in Sentinel-2 images in the context of the [COVID-19 Custom Script Contest](https://www.esa.int/Applications/Observing_the_Earth/Monitoring_trucks_and_trade_from_space) organized by ESA and Euro Data Cube.

The original Jupyter Notebook allowing the algorithm to be run on small areas is available in the Euro Data Cube [marketplace](https://eurodatacube.com/marketplace/notebooks/contributions/Detect_Trucks_Sentinel2.ipynb). The version presented in this Notebook has been optimised for the large-scale application of the workflow. 



In [ ]:
# Configure logging to stderr such that messages appear in notebook
# NOTE: Logging messages from python files which are called from the notebook 
#       will then also appear in the notebook.
import logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
)

## Before you start

### Requirements

In order to run the algorithm in this Jupyter Notebook the following subscriptions are needed:

- a subscription to EOxHub on the Euro Data Cube: Standard Plan (recommendation)
- a subscription to EDC Sentinel Hub on the Euro Data Cube: Enterprise L plan (recommendation). Please note that an Enterprise account is needed in order to support the [Batch API](https://docs.sentinel-hub.com/api/latest/api/batch/)
- access to a cloud storage bucket (this workflow is based on AWS S3) that has been [pre-configured](https://docs.sentinel-hub.com/api/latest/api/batch/#aws-s3-bucket-settings)


### Import libraries

The following libraries are pre-configured in your JupyterLab environement on the Euro Data Cube.

For this developement version, the Truck Detection library should be imported from the internal repository.

The `OSMPythonTools` is not installed by default on your Euro Data Cube workspace. If it hasn't yet been installed, the following cell will do.

In [2]:
# installations
import sys
import subprocess
def install_package(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
install_package("OSMPythonTools")

In [3]:
# Sentinel Hub
from sentinelhub import BBox

# Utilities
from os import environ
from shapely.geometry import mapping
from shapely.wkt import loads
import IPython.display
import yaml

# Truck detection algorithm
import truck_detection

# GeoDB
from xcube_geodb.core.geodb import GeoDBClient

## Credentials generation

### Sentinel Hub services
The credentials for Sentinel Hub services are automatically set by the Euro Data Cube libraries in the following cell.

In [4]:
# Fetch credentials as environement variables
#setup_environment_variables()
import os

# Pass Sentinel Hub credentials to dictionnary
sh_credentials = dict(client_id=os.environ["SH_CLIENT_ID"],
                      client_secret=os.environ["SH_CLIENT_SECRET"])

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`
* `SH_CLIENT_ID`, `SH_CLIENT_NAME`, `SH_CLIENT_SECRET`, `SH_INSTANCE_ID`



### Amazon AWS S3

The Batch Processing API needs to access an AWS S3 bucket to write the Sentinel-2 products (raw bands and derived products) needed to run the algorithm. 

For safety purposes, it is recommended to save a file named `.env` in the same directory as this notebook. The file should be structured in the following format:

```
user:
    AWS_CLIENT: "your-aws-client"
    AWS_SECRET: "your-aws-secret-key"
bucket:
    NAME: "your-aws-s3-bucket-name"
```
An example file is provided in the repository.

Please note that your AWS bucket should be located in `eu-central-1` and be configured for use with Sentinel Hub services (https://docs.sentinel-hub.com/api/latest/api/batch/#aws-s3-bucket-settings).

In [5]:
# Store in variables
AWS_CLIENT = os.environ["AWS_CLIENT"]
AWS_SECRET = os.environ["AWS_SECRET"]
S3_BUCKET = os.environ["BUCKET_NAME"]

## Set up input parameters


### 1) Area of Interest

In the following cell we will set the bounding box that covers the AOI and the projection code of the coordinates of the bounding box ([EPSG code](https://spatialreference.org/)).

In [26]:
# Specify a bounding box and projection code
bbox_aoi = 'POLYGON ((2.768549 42.410784, 2.769 42.410784, 2.769 42.411, 2.768549 42.411, 2.768549 42.410784))'

In [27]:
# Plot the AOI
IPython.display.GeoJSON(mapping(loads(bbox_aoi)))

<IPython.display.GeoJSON object>

### 2) Time period

In the following cell the time-period over which the algorithm will be run is defined. Furthermore a filter that allows to select which days of the week to be considered is also set. If the filter is not set, acquisitions on all days will be considered.

In [28]:
# Set overall time-period
time_period = ["2018-04-01", "2018-06-21"]

# Set a filter for days
week_days = ["Tuesday", "Wednesday", "Thursday"]

### 3) Road types

The following cell allows to filter the type of roads (defined by OSM) that are considered to contain trucks. 

In [29]:
# list of road types to be fetched, they have to be at least three 
osm_values = ['motorway', 'primary', 'trunk']

### 4) Cloud content

Specify the maximum cloud coverage (from Sentinel-2 tile metadata) used to return the Sentinel-2 scenes.

In [30]:
maxCC = 1

### 5) Output

In this version of the notebook a summary log file of the results is written the the specified output folder.

In [31]:
results_dir = "/home/jovyan/result-data"

# NOTE: tiny area for testing:
aoi = "POLYGON ((16.688232 47.815921, 16.688232 48.134017, 17.318573 48.134017, 17.318573 47.815921, 16.688232 47.815921))"
time_period = "2018-04-01/2018-05-01"
maxCC = 30
osm_values = ["motorway", "trunk"]
time_period = "2021-07-01/2021-08-31"
week_days = ["Monday", "Thursday"]

## Initialise Truck Detection class

The truck detection class is the main class containing all the parameters and functions to run the Truck detection algorithm.

In a first step we pass the input parameters (AOI, projection, time period, but also credentials) to the class.

In [32]:
# Initialise object
trucks = truck_detection.TruckDetector(aoi, 4326, time_period, output_folder=results_dir)

# Set credentials (inject them from EDC) for SH services
trucks.set_sh_credentials(**sh_credentials)

# Set credentials for AWS bucket (from aws_credentials file)
trucks.set_aws_credentials(AWS_CLIENT, AWS_SECRET, S3_BUCKET)

# Set allowed days for filtering
trucks.set_weekdays(week_days)

# Set maximum cloud cover allowed
trucks.filter["MaxCC"] = maxCC

## Fetch input data

Fetch all the input data needed to run the Truck Detection Algorithm

In a first step, the Sentinel-2 data are requested using Batch Process API.

In [33]:
trucks.get_sentinel_data()

Batch: f7e1da04-4b76-4d33-ba3d-2cecb3acac92 is CREATED.


8it [03:27, 25.92s/it]                       


Batch: f7e1da04-4b76-4d33-ba3d-2cecb3acac92 is DONE.


In [34]:
trucks.batchID

'f7e1da04-4b76-4d33-ba3d-2cecb3acac92'

In [35]:
# Check the batch status
trucks.batch_status()

DONE


### Option to run the algorithm on a previously 

To retrieve results from a previous Batch request run, please specify the `id` of the request and continue from here.

In [36]:
# trucks.batchID = 'Your-BatchID'

## Obtain tiles' information

In [37]:
# Get tiles' geometry, shape, transform, and crs information
trucks.get_tile_info()
trucks.tile_info

,tile,geometry,shape,transform
0,31TDG_8_0,"Geometry(POLYGON ((480000 4690000, 480000 4700...","(1000, 1000)","(10.0, 0.0, 480000.0, 0.0, -10.0, 4700000.0, 0..."
1,31TDG_9_0,"Geometry(POLYGON ((490000 4690000, 490000 4700...","(1000, 1000)","(10.0, 0.0, 490000.0, 0.0, -10.0, 4700000.0, 0..."
2,31TDH_8_9,"Geometry(POLYGON ((480000 4700000, 480000 4710...","(1000, 1000)","(10.0, 0.0, 480000.0, 0.0, -10.0, 4710000.0, 0..."
3,31TDH_9_9,"Geometry(POLYGON ((490000 4700000, 490000 4710...","(1000, 1000)","(10.0, 0.0, 490000.0, 0.0, -10.0, 4710000.0, 0..."


## Fetch OSM data

The OSM data is queried in a public geoDB. If there are no results returned the OSm API is triggered (for cases outside Europe).

In [41]:
# Get OSM data from GeoDB
trucks.get_osm_data(osm_values)
trucks.tile_info

,tile,geometry,shape,transform,CRS,OSM_roads
0,31TDG_8_0,"Geometry(POLYGON ((480000 4690000, 480000 4700...","(1000, 1000)","(10.0, 0.0, 480000.0, 0.0, -10.0, 4700000.0, 0...",EPSG:32631,[[POLYGON ((489490.8004198414 4695377.74955373...
1,31TDG_9_0,"Geometry(POLYGON ((490000 4690000, 490000 4700...","(1000, 1000)","(10.0, 0.0, 490000.0, 0.0, -10.0, 4700000.0, 0...",EPSG:32631,[[POLYGON ((490513.4100637027 4692458.43596756...
2,31TDH_8_9,"Geometry(POLYGON ((480000 4700000, 480000 4710...","(1000, 1000)","(10.0, 0.0, 480000.0, 0.0, -10.0, 4710000.0, 0...",EPSG:32631,[[POLYGON ((485260.8010099136 4708255.59896215...
3,31TDH_9_9,"Geometry(POLYGON ((490000 4700000, 490000 4710...","(1000, 1000)","(10.0, 0.0, 490000.0, 0.0, -10.0, 4710000.0, 0...",EPSG:32631,"[[], [], [POLYGON ((491271.7981786217 4709998...."


## Process by tiles and write the results to GeoDB

In [42]:
# Process by tiles and store the results to GeoDB
trucks.process_tiles(out_format="gpkg")

  0%|          | 0/4 [00:00<?, ?it/s]

Processing rows from 0 to 24
Processing rows from 0 to 29
Processing rows from 0 to 22
Processing rows from 0 to 33
Processing rows from 0 to 41


 25%|██▌       | 1/4 [00:06<00:20,  6.69s/it]

Processing rows from 0 to 17
Processing rows from 0 to 15
Processing rows from 0 to 14
Processing rows from 0 to 13
Processing rows from 0 to 42


 50%|█████     | 2/4 [00:12<00:12,  6.36s/it]

Processing rows from 0 to 76
Processing rows from 0 to 71
Processing rows from 0 to 69
Processing rows from 0 to 95


 75%|███████▌  | 3/4 [00:18<00:06,  6.05s/it]

Processing rows from 0 to 1


100%|██████████| 4/4 [00:21<00:00,  5.30s/it]


## Read the results from GeoDB

In [1]:
# reading from geodb is currently disabled because the out format is fixed to gpkg
#geodb = GeoDBClient()
#gdf_results = geodb.get_collection_pg(collection=trucks.geodb_collection)
#gdf_results